In [7]:
import os
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
import cv2
import json
from sklearn.utils import shuffle

In [8]:
def processName(text):
    if text[1]=="_":
        return (text[0]+".jpeg")
    elif text[2]=="_":
        return (text[:2]+".jpeg")
    else:
        return (text[:3]+".jpeg")

In [9]:
def annotate(file, dataset):
    df = pd.read_csv(file)
    df.drop_duplicates(keep="first", inplace=True)
    fileName = file.split(".")
    imageName = fileName[0] + "." + fileName[1]
    data = dataset[dataset["filename"]==imageName]
    df["Similarity"] = 100
    if len(data["Encoding"].to_list())>0:
        true = data["Encoding"].to_list()[0]
        for index in df.index:
            df["Similarity"][index] = abs(true - df["CandidatePosition"][index])
        df["Output"] = 0
        for index in df.index:
            if df["Similarity"][index] == df["Similarity"].min():
                df["Output"][index] = 1
        return df
    else:
        true = 0
        for index in df.index:
            df["Similarity"][index] = abs(true - df["CandidatePosition"][index])
        df["Output"] = 0
        return df


In [16]:
def process(datasetDirPath, Class):
    os.chdir(datasetDirPath)
    datasetDir = os.listdir(datasetDirPath)
    CI = []
    for file in datasetDir:
        df = pd.read_csv(file)
        for index in df.index:
            ci = []
            ci.append(df["x1"][index])
            ci.append(df["y1"][index])
            ci.append(df["x2"][index])
            ci.append(df["y2"][index])
            CI.append(ci)
    CIpca = PCA(n_components=1)
    CIpca.fit_transform(CI)
    for file in datasetDir:
        df = pd.read_csv(file)
        df["CandidatePosition"] = 0
        for index in df.index:
            ci = []
            ci.append(df["x1"][index])
            ci.append(df["y1"][index])
            ci.append(df["x2"][index])
            ci.append(df["y2"][index])
            df["CandidatePosition"][index] = CIpca.transform([ci])[0]
        df.to_csv(file)

    os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Dataset")
    dataset = pd.read_csv("_annotations.csv")
    dataset = dataset[dataset["class"]==Class]
    dataset["Encoding"] = 0
    os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Images")
    for index in dataset.index:
        dataset["filename"][index] = processName(dataset["filename"][index])
        img = cv2.imread(dataset["filename"][index])
        dataset["xmin"][index]/=img.shape[1]
        dataset["xmax"][index]/=img.shape[1]
        dataset["ymin"][index]/=img.shape[0]
        dataset["ymax"][index]/=img.shape[0]
        arr = []
        arr.append(dataset["xmin"][index])
        arr.append(dataset["ymin"][index])
        arr.append(dataset["xmax"][index])
        arr.append(dataset["ymax"][index])
        encoding = CIpca.transform([arr])
        dataset["Encoding"][index] = encoding
    os.chdir(datasetDirPath)
    for file in datasetDir:
        output = annotate(file, dataset)
        out1 = output[output["Output"]==1]
        out0 = output[output["Output"]==0]
        if out1.shape[0]==0:
            out1.to_csv(file)
        elif out0.shape[0]>3:
            out0 = shuffle(out0, random_state=30)
            out0 = out0.iloc[:3]
            df = pd.concat([out0, out1])
            df.to_csv(file)
        else:
            output.to_csv(file)


In [17]:
process("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/InvoiceDateDataset", "InvoiceDate")
process("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/TotalAmountDataset", "TotalAmount")
process("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/InvoiceNumberDataset", "InvoiceNumber")

/tmp/ipykernel_6135/3803922165.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["CandidatePosition"][index] = CIpca.transform([ci])[0]
/tmp/ipykernel_6135/3803922165.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["CandidatePosition"][index] = CIpca.transform([ci])[0]
/tmp/ipykernel_6135/3803922165.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["CandidatePosition"][index] = CIpca.transform([ci])[0]
/tmp/ipykerne

In [18]:
file = open("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Models/wordEmbeddings.json")
dict = json.load(file)

os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/InvoiceDateDataset")
datasets = os.listdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/InvoiceDateDataset")

dfs = []
for file in datasets:
    df = pd.read_csv(file)
    dfs.append(df)
DF = pd.concat(dfs)

IDdropCols = []
for col in dict.keys():
    if DF[col].max()==0 and DF[col].min()==0:
        IDdropCols.append(col)
        IDdropCols.append(col+"_X")
        IDdropCols.append(col+"_Y")

DF.drop(columns=IDdropCols, inplace=True)

valids = {}
for key in dict.keys():
    if key in DF.columns:
        valids[key] = 1
    else:
        valids[key] = 0

with open("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Models/IDVocab.json", "w") as file:
    json.dump(valids, file)

os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/TotalAmountDataset")
datasets = os.listdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/TotalAmountDataset")

dfs = []
for file in datasets:
    df = pd.read_csv(file)
    dfs.append(df)

DF = pd.concat(dfs)

TAdropCols = []
for col in dict.keys():
    if DF[col].max()==0 and DF[col].min()==0:
        TAdropCols.append(col)
        TAdropCols.append(col+"_X")
        TAdropCols.append(col+"_Y")

DF.drop(columns=TAdropCols, inplace=True)

valids = {}
for key in dict.keys():
    if key in DF.columns:
        valids[key] = 1
    else:
        valids[key] = 0

with open("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Models/TAVocab.json", "w") as file:
    json.dump(valids, file)

os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/InvoiceNumberDataset")
datasets = os.listdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/InvoiceNumberDataset")

dfs = []
for file in datasets:
    df = pd.read_csv(file)
    dfs.append(df)

DF = pd.concat(dfs)

INdropCols = []
for col in dict.keys():
    if DF[col].max()==0 and DF[col].min()==0:
        INdropCols.append(col)
        INdropCols.append(col+"_X")
        INdropCols.append(col+"_Y")

DF.drop(columns=INdropCols, inplace=True)

valids = {}
for key in dict.keys():
    if key in DF.columns:
        valids[key] = 1
    else:
        valids[key] = 0

with open("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Models/INVocab.json", "w") as file:
    json.dump(valids, file)

In [19]:
file = open("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Models/IDVocab.json")
IDVocab = json.load(file)
file = open("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Models/TAVocab.json")
TAVocab = json.load(file)
file = open("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Models/INVocab.json")
INVocab = json.load(file)
print("ID : ", sum(IDVocab.values()))
print("TA : ", sum(TAVocab.values()))
print("IN : ", sum(INVocab.values()))

ID :  799
TA :  555
IN :  1172
